In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date

In [ ]:
result_list = []
for num in range(1, 530):
    url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL'
    params = {
        'page' : num
    }
    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content)
    tr_list = soup.find_all('tr')[1:]
    for tr in tr_list:
        time = tr.find_all('td')[0].text.strip()
        a = time.split('.')
        time2 = date(int(a[0]), int(a[1]), int(a[2]))
        price = tr.find_all('td')[1].text
        
        new_dict = {
            'date' : time2,
            'price' : price
        }
        result_list.append(new_dict)

In [ ]:
df = pd.DataFrame(result_list)
df = df.set_index('date')
df.head()

In [ ]:
df.loc[date(2004, 12, 31):date(2004, 11, 25)]

In [ ]:
df.tail()

# 콜금리 상승/ 하락 판단  
 - 1개월전 금리와 비교하여 판단
 - 해당일이 존재하지 않을 경우 전날 가격으로 판단

### 판다스 DateOffset 모듈  
 - 알아서 다 해준다.

In [ ]:
day = date(2018, 1, 3)
a = day - pd.DateOffset(months = 1)
a

In [ ]:
df.loc[date(2005, 1, 3)]

In [ ]:
day = date(2005, 3, 31)
b = day - pd.DateOffset(months = 1)
print(b)
print(b.date())
df.loc[b.date()].price

In [ ]:
def up_down_call(df):
    df_ud = pd.DataFrame(index = df.index, columns = ['up_down'])
    for i in range(len(df.index)):
        time = df.index[i]
        time2 = df.index[i] - pd.DateOffset(months = 1)
        
        if time.year >= 2005:

            while time2.date() not in df.index:
                time2 = time2 - pd.DateOffset(days = 1)
                
            if time2.date() in df.index:

                if df.loc[time].price > df.loc[time2.date()].price:
                    df_ud['up_down'][i] = '상승'
                elif df.loc[time].price == df.loc[time2.date()].price:
                    df_ud['up_down'][i] = 0
                else:
                    df_ud['up_down'][i] = '하락'
    return df_ud

In [ ]:
df_t = up_down_call(df)
df_t.head()

In [ ]:
df_a = df.join(df_t, on = 'date', how = 'outer')
df_call = df_a.loc[:date(2004, 10, 1)]

In [ ]:
df_call.tail()

In [ ]:
type(df_call.index[1])

# 엑셀파이로 저장

In [ ]:
import os

In [ ]:
BASE_DIR = os.getcwd()
SAVE_DB_DIR = os.path.join(BASE_DIR, 'call')

if not os.path.exists(SAVE_DB_DIR):
    os.makedirs(SAVE_DB_DIR)

#텍스트 파일 만들기
temp_name = f"call_os.xlsx"
txt_name = os.path.join(SAVE_DB_DIR, temp_name)
excel_writer = pd.ExcelWriter(txt_name, engine = 'xlsxwriter')
#     df_report.to_excel(excel_writer, sheet_name='report')
df_call.to_excel(excel_writer)

excel_writer.save()

In [ ]:
df_call.to_excel('call_pd.xlsx')

In [ ]:
data_pd = pd.read_excel('call_pd.xlsx')

In [ ]:
data_pd.head()

In [ ]:
df_call.tail()

In [ ]:
df_call.index[0]

In [ ]:
df_call.index[0] == data_pd.loc[0, 'date']

In [ ]:
data_pd.loc[0, 'date']

In [ ]:
data_pd['date'] = pd.to_datetime(data_pd['date']).apply(lambda x: x.date()).values

In [ ]:
data_pd.loc[0, 'date']

# 빈 날짜 채우기

In [ ]:
import pandas as pd
from datetime import date, timedelta

In [ ]:
date_data = pd.DataFrame(columns = ['date'])
start = date(2004, 10, 1)
end = date(2019, 7, 15)
for n in range(int((end - start).days) + 1):
    date_data.loc[n, 'date'] = start + timedelta(n)

In [ ]:
date_data.tail()

In [ ]:
date_data.loc[5308, 'date']

In [ ]:
import pandas as pd
import os

In [ ]:
cd call

In [ ]:
call = pd.read_excel('call_M.xlsx')

In [ ]:
call.head()

# timestamp형식을 timedelta로 바꾸기

In [ ]:
a = call.set_index('date')
a.index[1]

In [ ]:
type(date_data.loc[10, 'date'])

In [ ]:
type(call.loc[10, 'date'])

In [ ]:
call['date'] = pd.to_datetime(call['date']).apply(lambda x: x.date()).values

In [ ]:
import pandas as pd

In [ ]:
type(call.loc[10, 'date'])

In [ ]:
call.head()

# 두 데이터 합치기

In [ ]:
M = date_data.merge(call, on = 'date', how = 'outer')
M.head(10)

In [ ]:
M['price'] = M['price'].fillna(method='bfill')

In [ ]:
M.head()

In [ ]:
M.to_excel('콜금리만 채우기.xlsx')

In [ ]:
M['up_down'] = M['up_down'].fillna(method='bfill')

In [ ]:
M.head()

In [ ]:
M.to_excel('up_down도 채우기.xlsx')